## 0 - Setup and data preparation
This cell loads libraries, the dataset, performs preprocessing, and basic feature engineering before the custom features.

In [ ]:
# -*- coding: utf-8 -*-
"""Project.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1nCZClEDak_KN_ZUgPhhRQ3Bp8T3v4ywz

Task1-4
"""

# !pip install datasets
# !pip install python-dotenv

from dotenv import load_dotenv
import os
from huggingface_hub import login

load_dotenv()

hf_token = os.getenv("HF_TOKEN")
login(token=hf_token)

from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
dataset = load_dataset("KFUPM-JRCAI/arabic-generated-abstracts")
print(dataset)

# Inspect column names and data types for one split (e.g., 'by_polishing')
print("\nFeatures in 'by_polishing':")
print(dataset['by_polishing'].features)

# Check dataset info (shape, structure, statistics)
print("\nDataset info for 'by_polishing':")
print(dataset['by_polishing'])

# Choose one split (e.g., by_polishing)
split1 = dataset["by_polishing"]

# Count human-written abstracts
num_human = len(split1["original_abstract"])

# Count AI-generated abstracts (4 per row)
num_ai = len(split1["allam_generated_abstract"]) \
       + len(split1["jais_generated_abstract"]) \
       + len(split1["llama_generated_abstract"]) \
       + len(split1["openai_generated_abstract"])

print("Number of human abstracts:", num_human)
print("Number of AI-generated abstracts:", num_ai)

# Distribution ratio
total = num_human + num_ai
print("Human %:", round(num_human / total * 100, 2))
print("AI %:", round(num_ai / total * 100, 2))

import pandas as pd
# Convert to pandas for easier checks
df = pd.DataFrame(split1)

# 1. Missing values
print("Missing values per column:")
print(df.isnull().sum())
print("_________________________________________")

# 2. Duplicates
print("\nNumber of duplicate rows:", df.duplicated().sum())

# Also check duplicates in each column separately
for col in df.columns:
    print(f"Duplicates in column {col}: {df[col].duplicated().sum()}")
print("_________________________________________")


# 3. Inconsistencies: empty strings or only spaces
for col in df.columns:
    empty_count = df[col].apply(lambda x: str(x).strip() == "").sum()
    print(f"Empty/blank values in column {col}: {empty_count}")

split2 = dataset["from_title"]

# Count human-written abstracts
num_human = len(split2["original_abstract"])

# Count AI-generated abstracts (4 per row)
num_ai = len(split2["allam_generated_abstract"]) \
       + len(split2["jais_generated_abstract"]) \
       + len(split2["llama_generated_abstract"]) \
       + len(split2["openai_generated_abstract"])

print("Number of human abstracts:", num_human)
print("Number of AI-generated abstracts:", num_ai)

# Distribution ratio
total = num_human + num_ai
print("Human %:", round(num_human / total * 100, 2))
print("AI %:", round(num_ai / total * 100, 2))

import pandas as pd
# Convert to pandas for easier checks
df = pd.DataFrame(split2)

# 1. Missing values
print("Missing values per column:")
print(df.isnull().sum())
print("_________________________________________")

# 2. Duplicates
print("\nNumber of duplicate rows:", df.duplicated().sum())

# Also check duplicates in each column separately
for col in df.columns:
    print(f"Duplicates in column {col}: {df[col].duplicated().sum()}")
print("_________________________________________")


# 3. Inconsistencies: empty strings or only spaces
for col in df.columns:
    empty_count = df[col].apply(lambda x: str(x).strip() == "").sum()
    print(f"Empty/blank values in column {col}: {empty_count}")

split3 = dataset["from_title_and_content"]

# Count human-written abstracts
num_human = len(split3["original_abstract"])

# Count AI-generated abstracts (4 per row)
num_ai = len(split3["allam_generated_abstract"]) \
       + len(split3["jais_generated_abstract"]) \
       + len(split3["llama_generated_abstract"]) \
       + len(split3["openai_generated_abstract"])

print("Number of human abstracts:", num_human)
print("Number of AI-generated abstracts:", num_ai)

# Distribution ratio
total = num_human + num_ai
print("Human %:", round(num_human / total * 100, 2))
print("AI %:", round(num_ai / total * 100, 2))

import pandas as pd
# Convert to pandas for easier checks
df = pd.DataFrame(split3)

# 1. Missing values
print("Missing values per column:")
print(df.isnull().sum())
print("_________________________________________")

# 2. Duplicates
print("\nNumber of duplicate rows:", df.duplicated().sum())

# Also check duplicates in each column separately
for col in df.columns:
    print(f"Duplicates in column {col}: {df[col].duplicated().sum()}")
print("_________________________________________")


# 3. Inconsistencies: empty strings or only spaces
for col in df.columns:
    empty_count = df[col].apply(lambda x: str(x).strip() == "").sum()
    print(f"Empty/blank values in column {col}: {empty_count}")

import re
import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.isri import ISRIStemmer
from datasets import load_dataset

# Download required NLTK resources
nltk.download('stopwords')

# Check columns
print(df.head())

#Define Arabic text cleaning functions
# Remove tashkeel (diacritics)
def remove_diacritics(text):
    arabic_diacritics = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    return re.sub(arabic_diacritics, '', text)

# Normalize Arabic text
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "و", text)
    text = re.sub("ئ", "ي", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("[^؀-ۿ ]+", " ", text)  # remove non-Arabic chars
    return text

# Initialize stopwords and stemmer
arabic_stopwords = set(stopwords.words("arabic"))
stemmer = ISRIStemmer()

# Full preprocessing pipeline
def preprocess_text(text):
    text = str(text)
    text = remove_diacritics(text)
    text = normalize_arabic(text)
    tokens = text.split()
    tokens = [w for w in tokens if w not in arabic_stopwords]
    tokens = [stemmer.stem(w) for w in tokens]
    return " ".join(tokens)

# Apply preprocessing
text_columns = [
    'original_abstract',
    'allam_generated_abstract',
    'jais_generated_abstract',
    'llama_generated_abstract',
    'openai_generated_abstract'
]
for col in text_columns:
    clean_col = col + "_clean"
    df[clean_col] = df[col].apply(preprocess_text)
print(" Preprocessing complete! Here are the new columns:")
print(df.columns)
df.head(2)

# Task 2.2: Exploratory Data Analysis (EDA)
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from collections import Counter
import seaborn as sns
import numpy as np

# Combine AI abstracts into one column
ai_texts = pd.concat([
    df['allam_generated_abstract_clean'],
    df['jais_generated_abstract_clean'],
    df['llama_generated_abstract_clean'],
    df['openai_generated_abstract_clean']
], axis=0).dropna().tolist()

human_texts = df['original_abstract_clean'].dropna().tolist()

# --- Statistical Analysis ---
def text_stats(texts):
    words = [w for txt in texts for w in txt.split()]
    avg_word_len = np.mean([len(w) for w in words])
    avg_sent_len = np.mean([len(txt.split()) for txt in texts])
    vocab = set(words)
    ttr = len(vocab) / len(words)
    return avg_word_len, avg_sent_len, ttr

stats_human = text_stats(human_texts)
stats_ai = text_stats(ai_texts)

print("\n Statistical Summary:")
print(f"Human-written: Avg word len={stats_human[0]:.2f}, Avg sent len={stats_human[1]:.2f}, TTR={stats_human[2]:.3f}")
print(f"AI-generated : Avg word len={stats_ai[0]:.2f}, Avg sent len={stats_ai[1]:.2f}, TTR={stats_ai[2]:.3f}")

# --- N-gram Frequency ---
def plot_top_ngrams(texts, n=2, top_k=15):
    from sklearn.feature_extraction.text import CountVectorizer
    vec = CountVectorizer(ngram_range=(n, n))
    bag = vec.fit_transform(texts)
    sum_words = bag.sum(axis=0)
    freqs = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    freqs = sorted(freqs, key=lambda x: x[1], reverse=True)[:top_k]
    words, counts = zip(*freqs)
    plt.figure(figsize=(10,4))
    sns.barplot(x=list(counts), y=list(words))
    plt.title(f"Top {top_k} {n}-grams – {n}-grams for {'Human' if texts==human_texts else 'AI'} abstracts")
    plt.show()

print("\n🔤 Top Bigrams for Human-written abstracts:")
plot_top_ngrams(human_texts, n=2)

print("\n🔤 Top Bigrams for AI-generated abstracts:")
plot_top_ngrams(ai_texts, n=2)

import matplotlib.pyplot as plt

df["human_length"] = df["original_abstract"].apply(lambda x: len(x.split()))
df["ai_length"] = df["openai_generated_abstract"].apply(lambda x: len(x.split()))

plt.figure(figsize=(8,5))
plt.hist(df["human_length"], bins=30, alpha=0.6, label="Human-written", color='blue')
plt.hist(df["ai_length"], bins=30, alpha=0.6, label="AI-generated", color='orange')
plt.xlabel("Sentence Length (words)")
plt.ylabel("Frequency")
plt.title("Sentence Length Distribution")
plt.legend()
plt.show()

def type_token_ratio(text):
    words = text.split()
    return len(set(words)) / len(words) if words else 0

df["human_ttr"] = df["original_abstract"].apply(type_token_ratio)
df["ai_ttr"] = df["openai_generated_abstract"].apply(type_token_ratio)

plt.figure(figsize=(6,5))
plt.boxplot([df["human_ttr"], df["ai_ttr"]], labels=["Human", "AI"])
plt.title("Vocabulary Richness (Type–Token Ratio)")
plt.ylabel("TTR Score")
plt.show()

from collections import Counter
import pandas as pd

human_words = " ".join(df["original_abstract"]).split()
ai_words = " ".join(df["openai_generated_abstract"]).split()

human_freq = Counter(human_words)
ai_freq = Counter(ai_words)

common_words = set(list(human_freq.keys())[:100]) & set(list(ai_freq.keys())[:100])

data = []
for w in common_words:
    data.append((w, human_freq[w], ai_freq[w]))

freq_df = pd.DataFrame(data, columns=["word", "human", "ai"]).sort_values("human", ascending=False)[:15]

freq_df.plot(x="word", kind="bar", figsize=(10,5), title="Top Words: Human vs AI", rot=45)
plt.ylabel("Frequency")
plt.show()

# Combine all splits into one df_human
splits = ["by_polishing", "from_title", "from_title_and_content"]

df_human = pd.concat([dataset[s].to_pandas() for s in splits], ignore_index=True)

dfs = []

for split_name in ["by_polishing", "from_title", "from_title_and_content"]:
    split_df = dataset[split_name].to_pandas().copy()
    split_df["source_split"] = split_name   # <-- Create column manually
    dfs.append(split_df)

df_human = pd.concat(dfs, ignore_index=True)

ai_rows = []

for _, row in df_human.iterrows():
    ai_models = [
        ("allam", row["allam_generated_abstract"]),
        ("jais", row["jais_generated_abstract"]),
        ("llama", row["llama_generated_abstract"]),
        ("openai", row["openai_generated_abstract"]),
    ]

    for model_name, text in ai_models:
        ai_rows.append({
            "abstract_text": text,
            "source_split": row["source_split"],     # now this exists
            "generated_by": model_name,
            "label": 0  # AI
        })

# Convert to dataframe
df_ai = pd.DataFrame(ai_rows)

# Create human dataframe
df_h = pd.DataFrame({
    "abstract_text": df_human["original_abstract"],
    "source_split": df_human["source_split"],
    "generated_by": "human",
    "label": 1
})

# Final unified dataset
df = pd.concat([df_h, df_ai], ignore_index=True)

print("Final unified dataset shape:", df.shape)
df.head(10)

print(df.columns)

# Inspect column names and data types for one split (e.g., 'by_polishing')
print("\nFeatures in 'by_polishing':")
print(dataset['by_polishing'].features)

# Check dataset info (shape, structure, statistics)
print("\nDataset info for 'by_polishing':")
print(dataset['by_polishing'])

num_human = df[df["label"] == 1].shape[0]
num_ai = df[df["label"] == 0].shape[0]

total = num_human + num_ai

print("\n===== Target Variable Distribution =====")
print("Human-written abstracts:", num_human)
print("AI-generated abstracts:", num_ai)
print("Human %:", round(num_human / total * 100, 2))
print("AI %:", round(num_ai / total * 100, 2))

print("\n===== Missing Values =====")
print(df.isnull().sum())

print("\n===== Duplicate Rows =====")
print("Total duplicate rows:", df.duplicated().sum())

print("\n===== Duplicate values per column =====")
for col in df.columns:
    print(f"{col}: {df[col].duplicated().sum()}")

print("\n===== Empty / Blank Values =====")
for col in df.columns:
    empty_count = df[col].apply(lambda x: str(x).strip() == "").sum()
    print(f"{col}: {empty_count}")

# Check columns
print(df.head())

#Define Arabic text cleaning functions
# Remove tashkeel (diacritics)
def remove_diacritics(text):
    arabic_diacritics = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    return re.sub(arabic_diacritics, '', text)

# Normalize Arabic text
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "و", text)
    text = re.sub("ئ", "ي", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("[^؀-ۿ ]+", " ", text)  # remove non-Arabic chars
    return text

# Initialize stopwords and stemmer
arabic_stopwords = set(stopwords.words("arabic"))
stemmer = ISRIStemmer()

# Full preprocessing pipeline
def preprocess_text(text):
    text = str(text)
    text = remove_diacritics(text)
    text = normalize_arabic(text)
    tokens = text.split()
    tokens = [w for w in tokens if w not in arabic_stopwords]
    tokens = [stemmer.stem(w) for w in tokens]
    return " ".join(tokens)

# Apply preprocessing
text_columns = [
    'abstract_text',
    'source_split',
    'generated_by',
    'label',
]

# Apply preprocessing on the unified abstract text column
df["abstract_text_clean"] = df["abstract_text"].apply(preprocess_text)

print("Preprocessing complete! Here are the new columns:")
print(df.columns)

df.head(2)

"""**FEATURES:**"""

#important library
import re
import math
import numpy as np
import pandas as pd
import unicodedata
from collections import Counter
from datasets import load_dataset
import regex as re2

#Helper functions


def simple_word_tokenize(text):
    """
    Tokenize text into words / symbols with Arabic support.
    """
    return re2.findall(r"\p{Arabic}+|\w+|[^\s\w]", text, flags=re2.VERSION1)

def sentence_tokenize(text):
    """
    Split text into sentences using Arabic/English punctuation.
    """
    if not isinstance(text, str):
        return []
    parts = re.split(r'(?<=[\.\?\!\u061F\u061B])\s+', text)
    return [p.strip() for p in parts if p.strip()]

def paragraph_tokenize(text):
    """
    Split text into paragraphs based on double newlines.
    """
    if not isinstance(text, str):
        return []
    paragraphs = re.split(r'\s*\n\s*\n\s*|\s*\r\n\s*\r\n\s*', text.strip())
    return [p.strip() for p in paragraphs if p.strip()]

# Column names to use
original_text_columns = "abstract_text"
clean_text_columns = "abstract_text_clean"

# =============================
# 1. Tokens (use clean text)
# =============================
df["tokens"] = df[clean_text_columns].apply(
    lambda t: [tok for tok in simple_word_tokenize(t) if tok.strip()] if isinstance(t, str) else []
)

# =============================
# 2. Words (use clean tokens only)
# =============================
df["words"] = df["tokens"].apply(
    lambda toks: [tok for tok in toks if re.search(r'\w', tok)]
)

# =============================
# 3. Sentences (use original_text_columns for accurate sentence boundary detection)
# =============================
df["sentences"] = df[original_text_columns].apply(
    lambda t: sentence_tokenize(t)
)

# =============================
# 4. Paragraphs (use original_text_columns to preserve original structural breaks)
# =============================
df["paragraphs"] = df[original_text_columns].apply(
    lambda t: paragraph_tokenize(t)
)

print("Feature engineering completed! Columns now:")
print(df.columns)
df.head(2)

# Column names to use
original_text_columns = "abstract_text"
clean_text_columns = "abstract_text_clean"


## 1 - Feature 12: Number of short words / N

In [ ]:
# Feature 12: Number of short words / N
feature_name = f'{clean_text_columns}_f012_num_short_words_over_N'

def _short_word_ratio(words, max_length=2):
    """Calculates the ratio of words shorter than or equal to max_length to the total number of words."""
    total_words = len(words)
    if total_words == 0:
        return 0.0

    # Count words where length is less than or equal to the defined maximum
    short_words_count = sum(1 for w in words if len(w) <= max_length)

    return float(short_words_count) / total_words

df[feature_name] = df["words"].apply(_short_word_ratio)

df.head(5)


## 2 - Feature 35: Total number of paragraphs (P)

In [ ]:
# 35. Total number of paragraphs (P)
df['f035_Total_number_of_paragraphs_(P)'] = df["paragraphs"].apply(len)

df.head(5)


## 3 - Feature 58: Number of words among the top-100 most frequent words

In [ ]:
# 58. Number of words found in the 100 most frequent positions
# (approximation using global frequency over the clean words column)

from collections import Counter

# حساب الترددات على عمود الكلمات الموحد "words"
all_words = []
for words_list in df["words"]:
    if isinstance(words_list, list):
        all_words.extend(words_list)

counter = Counter(all_words)
top100_words = set([w for w, _ in counter.most_common(100)])  # Top 100 كلمات

# لحساب Feature 58
# ------------------------------
def count_top100_embedding_words(tokens, top_words_set=top100_words):
    if not isinstance(tokens, list):
        return 0
    return sum(1 for tok in tokens if tok in top_words_set)

# تطبيق Feature 58 على عمود الكلمات الموحد
feature_58_name = f'{clean_text_columns}_f058_num_top100_embedding_words'
df[feature_58_name] = df["words"].apply(count_top100_embedding_words)

# لفحص نتائج ميزة 58
df[["words", feature_58_name]].head()



In [ ]:
!pip install transformers -q


## 4 - Feature 81: Language-model-based LogRank score

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd

# تأكد إنك ثبّتت transformers في خلية فوق:
# !pip install transformers -q

model_name = "distilgpt2"  # نسخة أخف من gpt2
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def compute_logrank(text):
    if not isinstance(text, str) or not text.strip():
        return 0.0

    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=128  # قلّل الطول عشان السرعة
    )
    input_ids = inputs["input_ids"].to(device)

    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids)
        log_rank_score = outputs.loss.item()

    return log_rank_score

# نجهز العمود
feature_81_name = f"{clean_text_columns}_f081_logrank"

values = []
max_rows = 1000   # نحسب لأول 1000 عينة فقط، والباقي نخليه 0
for i, t in enumerate(df[clean_text_columns]):
    if i < max_rows and pd.notna(t):
        values.append(compute_logrank(t))
    else:
        values.append(0.0)

df[feature_81_name] = values

df[[feature_81_name]].head()

## 5 - Feature 104: Root Pattern Frequency

In [ ]:
# 104. Root Pattern Frequency
# يقيس استخدام أنماط الجذور العربية (مثل "فعل"، "مفعول") بطريقة مبسطة.

root_patterns = ["فعل", "افعال", "مفعول", "فاعلة"]  # مثال لأنماط جذور

def root_pattern_frequency(text):
    if not isinstance(text, str) or len(text.strip()) == 0:
        return 0.0
    count = sum(text.count(pattern) for pattern in root_patterns)
    total_words = len(simple_word_tokenize(text))
    return count / total_words if total_words > 0 else 0.0

feature_104_name = f"{original_text_columns}_f104_root_pattern_freq"
df[feature_104_name] = df[original_text_columns].apply(root_pattern_frequency)
df[[original_text_columns, feature_104_name]].head()


## 6 - Train/Validation/Test split and model training

In [ ]:
!pip -q install sentence-transformers xgboost

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

# First split: Train 70%, Temp 30%  (مع stratify عشان توزيع labels يبقى متوازن)
train_df, temp_df = train_test_split(
    df, test_size=0.30, random_state=42, shuffle=True, stratify=df["label"]
)

# Second split: Temp 30% -> 15% Val, 15% Test
val_df, test_df = train_test_split(
    temp_df, test_size=0.50, random_state=42, shuffle=True, stratify=temp_df["label"]
)

print("TOTAL:", len(df))
print("TRAIN:", len(train_df))
print("VAL:", len(val_df))
print("TEST:", len(test_df))

print("\nLabel distribution:")
print("TRAIN:\n", train_df["label"].value_counts(normalize=True))
print("VAL:\n", val_df["label"].value_counts(normalize=True))
print("TEST:\n", test_df["label"].value_counts(normalize=True))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorer = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1, 2),
    analyzer="word"
)

# Fit فقط على التدريب
tfidf_vectorer.fit(train_df["abstract_text_clean"])

# Transform
X_train_tfidf = tfidf_vectorer.transform(train_df["abstract_text_clean"])
X_val_tfidf   = tfidf_vectorer.transform(val_df["abstract_text_clean"])
X_test_tfidf  = tfidf_vectorer.transform(test_df["abstract_text_clean"])

print("TF-IDF shapes:")
print("Train:", X_train_tfidf.shape)
print("Validation:", X_val_tfidf.shape)
print("Test:", X_test_tfidf.shape)

In [ ]:
from scipy.sparse import hstack, csr_matrix

EXCLUDED_COLS = [
    "label", "abstract_text", "abstract_text_clean",
    "tokens", "words", "sentences", "paragraphs", "abstract_text_pos_tags"
]

# الأعمدة الرقمية فقط (الفيتشرز اللي طلعتها)
numeric_cols = [
    col for col in train_df.select_dtypes(include=np.number).columns
    if col not in EXCLUDED_COLS
]

X_train_num = csr_matrix(train_df[numeric_cols].to_numpy())
X_val_num   = csr_matrix(val_df[numeric_cols].to_numpy())
X_test_num  = csr_matrix(test_df[numeric_cols].to_numpy())

y_train = train_df["label"].to_numpy()
y_val   = val_df["label"].to_numpy()
y_test  = test_df["label"].to_numpy()

# دمج Sparse + Sparse (بدون مشاكل)
X_train = hstack([X_train_tfidf, X_train_num]).tocsr()
X_val   = hstack([X_val_tfidf,   X_val_num]).tocsr()
X_test  = hstack([X_test_tfidf,  X_test_num]).tocsr()

print("X and y are ready for ML models.")
print("Train:", X_train.shape, y_train.shape)
print("Validation:", X_val.shape, y_val.shape)
print("Test:", X_test.shape, y_test.shape)

print("\nNumeric feature count:", len(numeric_cols))

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt

lr_model = LogisticRegression(max_iter=2000, random_state=42, solver="liblinear")
lr_model.fit(X_train, y_train)

y_val_pred = lr_model.predict(X_val)
print("Validation Accuracy:", accuracy_score(y_val, y_val_pred))
print(classification_report(y_val, y_val_pred))

y_test_pred = lr_model.predict(X_test)
print("Test Accuracy:", accuracy_score(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred))

cm = confusion_matrix(y_test, y_test_pred)
plt.figure(figsize=(4,4))
plt.imshow(cm)
plt.title("Confusion Matrix - Logistic Regression")
plt.xlabel("Predicted")
plt.ylabel("Actual")
for (i, j), v in np.ndenumerate(cm):
    plt.text(j, i, str(v), ha="center", va="center")
plt.show()

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt

models = {}

# SVM (LinearSVC أسرع مع sparse)
svm_model = LinearSVC(C=1.0, random_state=42)
svm_model.fit(X_train, y_train)
models["SVM"] = svm_model

# Random Forest (قد يكون بطيء مع features كثيرة جدًا)
rf_model = RandomForestClassifier(
    n_estimators=200,
    random_state=42,
    n_jobs=-1
)
rf_model.fit(X_train, y_train)
models["RandomForest"] = rf_model

# XGBoost
xgb_model = xgb.XGBClassifier(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.1,
    eval_metric="logloss",
    random_state=42,
    n_jobs=-1
)
xgb_model.fit(X_train, y_train)
models["XGBoost"] = xgb_model

# Validation quick check
for name, model in models.items():
    yv = model.predict(X_val)
    print(f"\n{name} Validation Accuracy:", accuracy_score(y_val, yv))
    print(classification_report(y_val, yv))

# Test evaluation + confusion matrix
for name, model in models.items():
    yt = model.predict(X_test)
    print(f"\n===== {name} Test Evaluation =====")
    print("Accuracy:", accuracy_score(y_test, yt))
    print(classification_report(y_test, yt))

    cm = confusion_matrix(y_test, yt)
    plt.figure(figsize=(4,4))
    plt.imshow(cm)
    plt.title(f"Confusion Matrix - {name}")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    for (i, j), v in np.ndenumerate(cm):
        plt.text(j, i, str(v), ha="center", va="center")
    plt.show()

In [ ]:
from sentence_transformers import SentenceTransformer
from tensorflow.keras import layers, models
from sklearn.metrics import accuracy_score, classification_report

bert_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

X_train_emb = bert_model.encode(train_df["abstract_text_clean"].tolist(), convert_to_numpy=True)
X_val_emb   = bert_model.encode(val_df["abstract_text_clean"].tolist(), convert_to_numpy=True)
X_test_emb  = bert_model.encode(test_df["abstract_text_clean"].tolist(), convert_to_numpy=True)

y_train_nn = train_df["label"].to_numpy()
y_val_nn   = val_df["label"].to_numpy()
y_test_nn  = test_df["label"].to_numpy()

ffnn_model = models.Sequential([
    layers.Input(shape=(X_train_emb.shape[1],)),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(1, activation="sigmoid")
])

ffnn_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
ffnn_model.summary()

history = ffnn_model.fit(
    X_train_emb, y_train_nn,
    validation_data=(X_val_emb, y_val_nn),
    epochs=10,
    batch_size=32,
    verbose=1
)

# Predict (تصحيح: ravel)
y_test_pred_nn = (ffnn_model.predict(X_test_emb).ravel() > 0.5).astype(int)

print("FFNN Test Accuracy:", accuracy_score(y_test_nn, y_test_pred_nn))
print("Classification Report (FFNN Test):")
print(classification_report(y_test_nn, y_test_pred_nn))

In [ ]:
import os, joblib
from tensorflow.keras.models import Model as KerasModel

def save_all_models(models_dict, save_dir="models"):
    """
    Saves all ML/DL models to disk based on their type.

    Parameters:
    -----------
    models_dict : dict
        Example:
            {
                "logistic_regression": log_reg_model,
                "svm": svm_model,
                "random_forest": rf_model,
                "xgboost": xgb_model,
                "ffnn": ffnn_model
            }

    save_dir : str
        Directory where models will be saved.
    """

    # Create save folder
    os.makedirs(save_dir, exist_ok=True)

    for model_name, model_obj in models_dict.items():

        # Case 1 — Keras deep learning model
        if isinstance(model_obj, KerasModel):
            file_path = os.path.join(save_dir, f"{model_name}.h5")
            model_obj.save(file_path)
            print(f"[Saved] Keras model → {file_path}")

        # Case 2 — All pickle-compatible models (Sklearn, XGBoost)
        else:
            file_path = os.path.join(save_dir, f"{model_name}.pkl")
            joblib.dump(model_obj, file_path)
            print(f"[Saved] Pickle model → {file_path}")

    print("\nAll models saved successfully!")

In [ ]:
import os
import joblib
from tensorflow.keras.models import load_model as keras_load_model

def load_all_models(save_dir="models"):
    """
    Loads all ML/DL models from disk.

    Parameters:
    -----------
    save_dir : str
        Directory where models are saved.

    Returns:
    --------
    dict
        A dictionary containing the loaded models, with model names as keys.
    """
    loaded_models = {}
    if not os.path.exists(save_dir):
        print(f"Error: Directory '{save_dir}' not found.")
        return loaded_models

    for filename in os.listdir(save_dir):
        model_name, ext = os.path.splitext(filename)
        file_path = os.path.join(save_dir, filename)

        if ext == '.h5':  # Keras model
            try:
                loaded_models[model_name] = keras_load_model(file_path)
                print(f"[Loaded] Keras model → {file_path}")
            except Exception as e:
                print(f"Error loading Keras model {filename}: {e}")
        elif ext == '.pkl':  # Pickle-compatible model (Sklearn, XGBoost)
            try:
                loaded_models[model_name] = joblib.load(file_path)
                print(f"[Loaded] Pickle model → {file_path}")
            except Exception as e:
                print(f"Error loading pickle model {filename}: {e}")
        else:
            print(f"Skipping unknown file type: {filename}")

    print("\nAll models loaded successfully!")
    return loaded_models

# Load all models
loaded_models = load_all_models()


# lr_model_loaded = loaded_models['lr_model']
# svm_model_loaded = loaded_models['svm']
# rf_model_loaded = loaded_models['random_forest']
# xgb_model_loaded = loaded_models['xgboost']
# ffnn_model_loaded = loaded_models['ffnn']


### Verification: List Loaded Models



In [ ]:
print("Models successfully loaded:")
for model_name in loaded_models.keys():
    print(f"- {model_name}")
